<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-файл-с-данными-и-изучите-его" data-toc-modified-id="Откройте-файл-с-данными-и-изучите-его-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте файл с данными и изучите его</a></span></li><li><span><a href="#Разделите-исходные-данные-на-обучающую,-валидационную-и-тестовую-выборки." data-toc-modified-id="Разделите-исходные-данные-на-обучающую,-валидационную-и-тестовую-выборки.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделите исходные данные на обучающую, валидационную и тестовую выборки.</a></span></li><li><span><a href="#Исследуйте-качество-разных-моделей" data-toc-modified-id="Исследуйте-качество-разных-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте качество разных моделей</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверьте-качество-модели-на-тестовой-выборке" data-toc-modified-id="Проверьте-качество-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте качество модели на тестовой выборке</a></span></li><li><span><a href="#Проверьте-модели-на-вменяемость" data-toc-modified-id="Проверьте-модели-на-вменяемость-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверьте модели на вменяемость</a></span></li></ul></div>

## Откройте файл с данными и изучите его

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression 

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
df.duplicated().sum()

0

Предобработка данных уже была проведена. В таблице нет пропусков, нет дубликатов, названия столбцов оформлены корректно.

## Разделите исходные данные на обучающую, валидационную и тестовую выборки.

Необходимо разбить исходные данные в соотношении 3:1:1, где 3 части приходятся на обучающую выборку, 1 часть на валидационную и 1 часть на тестовую.

In [8]:
features = df.drop(['is_ultra'], axis=1)

In [9]:
target = df['is_ultra']

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345
)

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345
)

In [12]:
# проверим разделение на выборки
print(features_train.shape, features_valid.shape, features_test.shape,
     target_train.shape, target_valid.shape, target_test.shape)

(1928, 4) (643, 4) (643, 4) (1928,) (643,) (643,)


Исходные данные разбиты корректно.

## Исследуйте качество разных моделей

### Решающее дерево

In [13]:
best_model_dt = None
best_accuracy_dt = 0
best_depth_dt = 0
for depth in range(1, 6):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_train, target_train)
    predictions_dt = model_dt.predict(features_valid)
    accuracy_dt = accuracy_score(predictions_dt, target_valid)
    if accuracy_dt > best_accuracy_dt:
        best_model_dt = model_dt
        best_accuracy_dt = accuracy_dt
        best_depth_dt = depth

print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy_dt, 'Глубина дерева:', best_depth_dt)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 Глубина дерева: 3


### Случайный лес

In [14]:
best_model_rf = None
best_accuracy_rf = 0
best_depth_rf = 0
best_est_rf = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rf.fit(features_train, target_train)
        predictions_rf = model_rf.predict(features_valid)
        accuracy_rf = accuracy_score(predictions_rf, target_valid)
        if accuracy_rf > best_accuracy_rf:
            best_model_rf = model_rf
            best_accuracy_rf = accuracy_rf
            best_depth_rf = depth
            best_est_rf = est
            
print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy_rf,\
      'Глубина дерева:', best_depth_rf, 'Количество деревьев:', best_est_rf)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Глубина дерева: 8 Количество деревьев: 40


### Логистическая регрессия

In [15]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_lr, target_valid)
print('Accuracy модели логистической регрессии на валидационной выборке:', accuracy_lr)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


### Вывод

Метрика, отражающая долю правильных ответов (accuracy) лучше всего у модели Случайный лес с глубиной дерева 8 и количеством деревьев 40.

## Проверьте качество модели на тестовой выборке

In [16]:
# проверим наилучшую модель,
# полученную в предыдущем пункте
test_predictions = best_model_rf.predict(features_test)

In [17]:
test_accuracy = accuracy_score(test_predictions, target_test)

In [18]:
print('Accuracy наилучшей модели Случайный лес на тестовой выборке:', test_accuracy)

Accuracy наилучшей модели Случайный лес на тестовой выборке: 0.7962674961119751


Доля правильных ответов на тестовой выборке равна 0.796. Это немного меньше, чем на валидационной выборке 0.808.

## Проверьте модели на вменяемость

Чтобы проверить модель на вменяемость, необходимо сравнить ее со случайной, т.е. сравнить значение accuracy c 0.5. В нашем случае оно больше 0.5, значит модель вменяема.